In [ ]:
# Step 1: Install Required Dependencies
%pip install langchain
%pip install langgraph
%pip install langchain-openai
%pip install chromadb
%pip install python-dotenv

In [ ]:
# Step 2: Import Core Libraries and Configure Environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END

load_dotenv(dotenv_path='env.txt')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
embeddings = OpenAIEmbeddings()

In [ ]:
# Step 3: Build the Basic Agent
# Define Agent State with memory placeholders
class AgentState(TypedDict):
    """State container for agent memory and messages"""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    working_memory: dict  # Short-term context
    episodic_recall: list  # Retrieved past experiences
    semantic_facts: dict  # Retrieved knowledge

# Initialize vector store for future memory storage
vector_store = Chroma(
    collection_name="agent_memory",
    embedding_function=embeddings,
    persist_directory="./memory_store"
)

# Create base prompt and chain
base_prompt = PromptTemplate.from_template("""
You are a helpful assistant with memory capabilities.

Current conversation:
{messages}

Please respond to the latest message.
""")

output_parser = StrOutputParser()

# Define agent node
def agent_node(state: AgentState) -> dict:
    """Core agent logic - processes messages and generates responses"""
    messages = state["messages"][-5:] if state["messages"] else []
    formatted_messages = "\n".join([
        f"{msg.type}: {msg.content}" 
        for msg in messages
    ])
    
    chain = base_prompt | llm | output_parser
    response = chain.invoke({"messages": formatted_messages})
    
    return {"messages": [("assistant", response)]}

# Build and compile the graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.set_entry_point("agent")
workflow.add_edge("agent", END)

app = workflow.compile()

In [15]:
# Step 4: Test the Basic Agent
# Test the agent
test_input = {
    "messages": [("user", "Hello! What's the capital of France?")]
}

result = app.invoke(test_input)
print(result["messages"][-1].content)

Hello! The capital of France is Paris. How can I assist you further?
